<h1><center>DNN Model</center></h1> 


In [ ]:
#Data Proccesing
import pandas as pd
#Model
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense,add
from tensorflow.keras.models import Model

Project_Path='/Users/nickkarras/PycharmProjects/ParkingViolationPredictionGraph_Git'

### Dataset Split

In [ ]:
Distance_Data=pd.read_csv(Project_Path+ '/Data/Distance.csv',sep=',',index_col=0)
Final_Weather_Data=pd.read_csv(Project_Path+ '/Data/Final_Weather_Data.csv',low_memory=False,sep=',',index_col=0)
Legal_illegal=pd.read_csv(Project_Path+ '/Data/Scan_Data_Reg_2.3.csv',sep=',',index_col=0)

Legal_illegal=Legal_illegal.loc[Legal_illegal['Key'] < '2020-01-01']
Legal_illegal=Legal_illegal.dropna()
Legal_illegal.to_csv(Project_Path+ '/Data/Full_TimeSeries.csv')

train_data=Legal_illegal.loc[Legal_illegal['Key'] < '2019-10-01']
test_data=Legal_illegal.loc[Legal_illegal['Key'] > '2019-10-01']

train_data.to_csv(Project_Path+ '/Data/Train_TimeSeries.csv')
test_data.to_csv(Project_Path+ '/Data/Test_TimeSeries.csv')

### Data Preprocessing 

In [ ]:
def Scaller(Train,Test):
    Standar_Scaller = StandardScaler()
    Scalled_Train_data=Standar_Scaller.fit_transform(Train)
    Scalled_val_data = Standar_Scaller.transform(Test)
    return Scalled_Train_data,Scalled_val_data

In [ ]:
def Prepare_Dataset(Dataset): 
    Dataset=Dataset.drop(['Time_Int'], axis=1)
    a=Dataset['Slot_Timeint']
    b=Dataset['Ilegality_Rate']
    Dataset=Dataset.drop(['Slot_Timeint'], axis=1)
    Dataset=Dataset.drop(['Ilegality_Rate'], axis=1)
    Dataset.insert(8, "Real_Time", a, True)
    Dataset.insert(9, "Real_Rate", b, True)
    
    Dataset=pd.merge(Dataset, Final_Weather_Data, on='Key')
    Dataset=Dataset.drop(['Key'], axis=1)
    
    Dataset=pd.merge(Dataset, Distance_Data, on='Slot_id')
    Dataset=Dataset.drop(['Slot_id'], axis=1)
    return Dataset

In [ ]:
train_data=pd.read_csv(Project_Path+ '/Data/Train_TimeSeries.csv',sep=',',index_col=0)
test_data=pd.read_csv(Project_Path+ '/Data/Test_TimeSeries.csv',sep=',',index_col=0)

TestDF=Prepare_Dataset(test_data)
TrainDF=Prepare_Dataset(train_data)

train_targets=TrainDF['Real_Rate']
train_data=TrainDF.drop(['Real_Rate'], axis=1)
test_targets=TestDF['Real_Rate']
test_data=TestDF.drop(['Real_Rate'], axis=1)

train_data,test_data=Scaller(train_data,test_data)

### DNN Setup

In [ ]:

def Residual_NN(Input_Shape,Bias_Sigmoid_Initializer,Kernel_Relu_Initializer,Kernel_Sigmoid_Initializer):
    Input_Layer = Input(shape=(Input_Shape,))
    Dense_Layer1 = Dense(256, activation='relu',kernel_initializer=Kernel_Relu_Initializer)(Input_Layer)
    Dense_Layer2 = Dense(128, activation='relu',kernel_initializer=Kernel_Relu_Initializer)(Dense_Layer1)
    Dense_Layer3 = Dense(64, activation='relu',kernel_initializer=Kernel_Relu_Initializer)(Dense_Layer2)
    Dense_Layer4 = Dense(32, activation='relu',kernel_initializer=Kernel_Relu_Initializer)(Dense_Layer3)
    Dense_Layer5 = Dense(64, activation='relu',kernel_initializer=Kernel_Relu_Initializer)(Dense_Layer4)
    Residual_Add = add([Dense_Layer3, Dense_Layer5])
    Dense_Layer6 = Dense(16, activation='relu',kernel_initializer=Kernel_Relu_Initializer)(Residual_Add)
    Output_Layer = Dense(1, activation='sigmoid',kernel_initializer=Kernel_Sigmoid_Initializer,bias_initializer=Bias_Sigmoid_Initializer)(Dense_Layer6)
    
    model2 = Model(inputs=Input_Layer, outputs=Output_Layer)
    model2.compile(optimizer='adamax', loss='mse', metrics='mae')
    return model2

In [ ]:
def scheduler(epoch, lr):
    if epoch < 8:
        return lr
    else:
        return lr * tf.math.exp(-0.25)

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
Bias_Initial_Out=train_targets.mean()
Bias_Initializer=tf.keras.initializers.Constant(Bias_Initial_Out)
Kernel_Sigmoid_Initializer = tf.keras.initializers.GlorotUniform()
Kernel_Relu_Initializer =tf.keras.initializers.HeUniform()

## Training procces

Εκπαιδεύω για 10 εποχές

In [ ]:
model = Residual_NN(train_data.shape[1],Bias_Initializer,Kernel_Relu_Initializer,Kernel_Sigmoid_Initializer)
model.fit(train_data,train_targets,epochs=10, batch_size=16, verbose=1)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
model.save(Project_Path + '/DNN_Regressor')